# Aggregation Mini Challenge 2

Apply what you have learned so far about the aggregation framework to try solve this mini challenge

In [1]:
# Import pymongo dependencies
from pymongo import MongoClient

In [2]:
# Set up your MongoClient and database variable
client = MongoClient("mongodb://localhost:27017/")
db = client["aggregation_test"]

In [3]:
# Function which prints out all the results of a cursor
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

In [4]:
### Challenge Part 1:
###   Find the top sellers (by user name) based on the quantity of all prodcuts they have sold from the 'orders' collection.
###   Sort the sellers from highest to lowest quantity sold, and output the results to a new collection called 'top_sellers'
###
### Example entry from new collection: 
### {"_id": ObjectId(...), "seller_name": "Sarah", "num_items_sold": 7}

out_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group": {"_id": "$items.product_id", "total_quantity": {"$sum": "$items.quantity"}}},
    {"$lookup": 
        {"from": "products",
         "localField": "_id",
         "foreignField": "_id",
         "as": "products"
    }},
    {"$group": {"_id": {"$first": "$products.seller_id"}, "num_items_sold": {"$sum": "$total_quantity"}}},
    {"$lookup": 
        {"from": "users",
         "localField": "_id",
         "foreignField": "_id",
         "as": "sellers"
    }},
    {"$project": {"seller_name": {"$first": "$sellers.name"}, "num_items_sold": 1}},
    {"$sort": {"num_items_sold": -1}},
    {"$out": {"db": "aggregation_test","coll": "top_sellers"}} 
])

In [5]:
# Grab the results from your database
# print_cursor(out_cursor)
top_sellers_cursor = db.top_sellers.find({})
print_cursor(top_sellers_cursor)

{'_id': ObjectId('61b55563182b471298c07c2f'), 'num_items_sold': 7, 'seller_name': 'Sarah'}

{'_id': ObjectId('61b55563182b471298c07c30'), 'num_items_sold': 5, 'seller_name': 'Bob'}

{'_id': ObjectId('61b55563182b471298c07c33'), 'num_items_sold': 2, 'seller_name': 'Jessica'}

{'_id': ObjectId('61b55563182b471298c07c32'), 'num_items_sold': 1, 'seller_name': 'Lisa'}



In [11]:
### Challenge Part 2:
###   You want to reward your sellers with an extra day of a premium membership for each item they've sold. Add
###   an attribute called 'premium_exp_date' to the entries in the 'users' collection which contains the date their 
###   premium membership will expire. Currently no users have a premium membership, so use today as the start date
###   from their premium membership, but make sure your pipeline is able to add the extra days of premium to an
###   existing date if one existed. 
###
### Example entry from users collection: 
###   {'_id': ObjectId(...), 'name': 'Sarah', 'premium_exp_date': datetime.datetime(2021, 12, 24, 4, 2, 4, 146000)}

merge_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group": {"_id": "$items.product_id", "total_quantity": {"$sum": "$items.quantity"}}},
    {"$lookup": 
        {"from": "products",
         "localField": "_id",
         "foreignField": "_id",
         "as": "products"
    }},
    {"$group": {"_id": {"$first": "$products.seller_id"}, "num_items_sold": {"$sum": "$total_quantity"}}},
    {"$lookup": 
        {"from": "users",
         "localField": "_id",
         "foreignField": "_id",
         "as": "sellers"
    }},
    {"$project": {"_id": 1, "premium_exp_date": {
        "$switch": { "branches": [
            {"case": {"$eq": [{"$type" : {"$first": "$sellers.premium_exp_date"}}, 'missing']}, 
             "then": {"$dateAdd": {"startDate": "$$NOW", "unit": "day", "amount": "$num_items_sold"}}},
            {"case": {"$lte": [{"$first": "$sellers.premium_exp_date"}, "$$NOW"]}, 
             "then": {"$dateAdd": {"startDate": "$$NOW", "unit": "day", "amount": "$num_items_sold"}}},
            {"case": {"$gt": [{"$first": "$sellers.premium_exp_date"}, "$$NOW"]}, 
             "then": {"$dateAdd": {"startDate": {"$first": "$sellers.premium_exp_date"}, "unit": "day", "amount": "$num_items_sold"}}}
            ]
        }
    } }},
    {"$merge": {
        "into": {"db": "aggregation_test","coll": "users"},
        "on": "_id", # must specify a indexed attribute
        "whenMatched": "merge",
        "whenNotMatched":  "fail"
   }} 
])


In [12]:
# Print out your results
# print_cursor(merge_cursor)
users_cursor = db.users.find({})
print_cursor(users_cursor)

{'_id': ObjectId('61b55563182b471298c07c2f'), 'name': 'Sarah', 'premium_exp_date': datetime.datetime(2021, 12, 24, 21, 27, 19, 783000)}

{'_id': ObjectId('61b55563182b471298c07c30'), 'name': 'Bob', 'premium_exp_date': datetime.datetime(2021, 12, 22, 21, 27, 19, 783000)}

{'_id': ObjectId('61b55563182b471298c07c31'), 'name': 'Jose'}

{'_id': ObjectId('61b55563182b471298c07c32'), 'name': 'Lisa', 'premium_exp_date': datetime.datetime(2021, 12, 18, 21, 27, 19, 783000)}

{'_id': ObjectId('61b55563182b471298c07c33'), 'name': 'Jessica', 'premium_exp_date': datetime.datetime(2021, 12, 19, 21, 27, 19, 783000)}

{'_id': ObjectId('61b55563182b471298c07c34'), 'name': 'Tina'}



In [8]:
db.users.update_many({}, {"$unset": {"premium_exp_date": True}})